# Fact Manutenção

Tabela fato de manutenção com cálculo de downtime.

In [ ]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {fqtn(SCHEMA_GOLD, 'fact_manutencao')} (
    maintenance_sk STRING,
    equipment_id STRING,
    start_date_key INT,
    maintenance_type STRING,
    actual_start TIMESTAMP,
    actual_end TIMESTAMP,
    downtime_minutes DOUBLE,
    technician_id STRING,
    cost_estimate DOUBLE
) USING DELTA
PARTITIONED BY (start_date_key)
""")

spark.sql(f"""
INSERT OVERWRITE {fqtn(SCHEMA_GOLD, 'fact_manutencao')}
SELECT 
    maintenance_order_id as maintenance_sk,
    equipment_id,
    cast(date_format(actual_start, 'yyyyMMdd') as int) as start_date_key,
    maintenance_type,
    actual_start,
    actual_end,
    (unix_timestamp(actual_end) - unix_timestamp(actual_start))/60 as downtime_minutes,
    technician_id,
    ((unix_timestamp(actual_end) - unix_timestamp(actual_start))/3600) * 100 as cost_estimate
FROM {fqtn(SCHEMA_SILVER, 'maintenance_orders_clean')}
WHERE actual_start IS NOT NULL
""")